In [1]:
pip install -U bitsandbytes transformers peft accelerate trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 84.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: tokenizers
    Found existing inst

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2" # to try HuggingFaceH4/zephyr-7b-beta
dataset_name = "/kaggle/input/haludetect-dataset-1/finetune_data.csv"
# new_model = "zephyr-7b-beta-llamini"

In [8]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [9]:
#Adding the adapters in the layers
# prepare_model_for_kbit_training wraps the entire protocol for preparing a model before running a training
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
# get_peft_model returns a Peft model object from a model and a config.
model = get_peft_model(model, peft_config)

In [10]:
import pandas as pd
data = pd.read_csv(dataset_name)
data.head()

,Unnamed: 0,prompt,IsHallucination
0,0,An LLM was given the below Instructions to be ...,Hallucination
1,1,An LLM was given the below Instructions to be ...,Not Hallucination
2,2,An LLM was given the below Instructions to be ...,Hallucination
3,3,An LLM was given the below Instructions to be ...,Hallucination
4,4,An LLM was given the below Instructions to be ...,Hallucination


In [11]:
def format_text(row):
    return f'<s>[INST]{row["prompt"]}[/INST] {row["IsHallucination"]}</s>'

# Apply the custom function to each row and create a new column 'formatted_text'
data['formatted_text'] = data.apply(format_text, axis=1)

In [12]:
data.head()

,Unnamed: 0,prompt,IsHallucination,formatted_text
0,0,An LLM was given the below Instructions to be ...,Hallucination,<s>[INST]An LLM was given the below Instructio...
1,1,An LLM was given the below Instructions to be ...,Not Hallucination,<s>[INST]An LLM was given the below Instructio...
2,2,An LLM was given the below Instructions to be ...,Hallucination,<s>[INST]An LLM was given the below Instructio...
3,3,An LLM was given the below Instructions to be ...,Hallucination,<s>[INST]An LLM was given the below Instructio...
4,4,An LLM was given the below Instructions to be ...,Hallucination,<s>[INST]An LLM was given the below Instructio...


In [13]:
data = data.sample(frac = 1)
trimmed_data = data.head(1000)

In [14]:
trimmed_data.to_csv('Trimmed10k_Formatted_haludetect_data.csv')

In [15]:
final_data_path = "/kaggle/working/Trimmed10k_Formatted_haludetect_data.csv"

In [16]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=[final_data_path])

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
dataset = dataset['train']

In [18]:
dataset

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'prompt', 'IsHallucination', 'formatted_text'],
    num_rows: 1000
})

In [19]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to='none'
)


In [20]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="formatted_text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [21]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.740300
50,0.925700
75,1.611400
100,0.833100
125,1.558000
150,0.738100
175,1.640800
200,0.733400
225,1.493400
250,0.741500


TrainOutput(global_step=250, training_loss=1.2015599517822266, metrics={'train_runtime': 3628.2983, 'train_samples_per_second': 0.276, 'train_steps_per_second': 0.069, 'total_flos': 2.4377497236013056e+16, 'train_loss': 1.2015599517822266, 'epoch': 1.0})

In [25]:
new_model = 'halu_detect_v1'

In [23]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [24]:
trainer.model.push_to_hub(new_model, use_temp_dir=False)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/krobo224/halu_detect_v1/commit/f7cda8367ab36219a821129765e3e303993b4c5b', commit_message='Upload model', commit_description='', oid='f7cda8367ab36219a821129765e3e303993b4c5b', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
val_data = pd.read_csv('/kaggle/input/shroom-data/Shroom_Val_Data.csv')

In [27]:
val_data.shape

(499, 4)

In [31]:
import re

In [37]:
final_output = []
accuracy = 0
for index, row in val_data.iterrows():
    prompt = row['prompt']
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    output = result[0]['generated_text']
    match = re.search(r'\[/INST\] \[/INST\] [A-Za-z]+\.', output)
    
    if match:
        print(output)
        output = match.group(0).split()[-1][:-1]
        output = output.replace('[/INST] [/INST] ', '')
        output = output.replace('.', '')
    
    final_output.append(output)
    
    if row['IsHallucination'] == 'Not Hallucination' and 'Not Hallucination' in output:
        accuracy = accuracy + 1
    if row['IsHallucination'] == 'Hallucination' and 'Hallucination' in output and 'Not Hallucination' not in output:
        accuracy = accuracy + 1
#     if output != 'Not Hallucination' or output != 'Hallucination':
# #         print(output)

    
accuracy = accuracy/499
print(accuracy)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

<s>[INST] <s>[INST]An LLM was given the below Instructions to be performed on the Source Data
                            Instructions:- Provide a concise and clear definition for the word/ phrase enclosed in the <define> and </define> tags. Ensure that the definition is accurate and suitable for a general audience.
                            Source Data:- The writer had just entered into his eighteenth year , when he met at the table of a certain Anglo - Germanist an individual , apparently somewhat under thirty , of middle stature , a thin and <define> weaselly </define> figure , a sallow complexion , a certain obliquity of vision , and a large pair of spectacles .
                            
                            Below is the Generated output
                            Generated output:-Resembling or characteristic of a weasel.
                            
                            And we also have the Desired output for the task:-
                            Desired Outp

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

<s>[INST] <s>[INST]An LLM was given the below Instructions to be performed on the Source Data
                            Instructions:- Provide a concise and clear definition for the word/ phrase enclosed in the <define> and </define> tags. Ensure that the definition is accurate and suitable for a general audience.
                            Source Data:- Sailors ' and fishermen 's <define> sheath - knives </define> . Sailors ' sheath - knives .   Wilcox , Crittenden & Co. , Middletown , Conn.29428 .   Sheath and belt , with " law - abiding " sheath - knife . First quality.29426 .   Sheath and belt . Second quality.29427 .   Sheath and belt , with " law - abiding " sheath - knife . Third quality . The " law - abiding " sheath - knife is round at the tip of the blade , which is also thick and dull .
                            
                            Below is the Generated output
                            Generated output:-Alternative form of sheath knife
                      

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

...

<s>[INST] <s>[INST]An LLM was given the below Instructions to be performed on the Source Data
                            Instructions:- Translate the below text:-
                            Source Data:- Не думаю, что Том будет помогать нам это делать.
                            
                            Below is the Generated output
                            Generated output:-I don't think Tom's gonna help us do this.
                            
                            And we also have the Desired output for the task:-
                            Desired Output:-I don't think Tom is going to help us do that.
                            
                            Now tell if the Generated output is hallucinated or not, respond with just hallucination/ not hallucination

                        [/INST] [/INST] Hallucination. The Generated output is not grammatically correct.


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

<s>[INST] <s>[INST]An LLM was given the below Instructions to be performed on the Source Data
                            Instructions:- Translate the below text:-
                            Source Data:- Россия превратилась во вторую сверхдержаву.
                            
                            Below is the Generated output
                            Generated output:-Russia has turned into a second superpower.
                            
                            And we also have the Desired output for the task:-
                            Desired Output:-Russia had emerged as a second superpower.
                            
                            Now tell if the Generated output is hallucinated or not, respond with just hallucination/ not hallucination

                        [/INST] [/INST] Hallucination. The Generated output is not grammatically correct. The correct form would be "had emerged" instead of "had turned into" and "second superpower" instead of "s

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

<s>[INST] <s>[INST]An LLM was given the below Instructions to be performed on the Source Data
                            Instructions:- Translate the below text:-
                            Source Data:- Дети получили подарки.
                            
                            Below is the Generated output
                            Generated output:-The children received gifts.
                            
                            And we also have the Desired output for the task:-
                            Desired Output:-The children got some presents.
                            
                            Now tell if the Generated output is hallucinated or not, respond with just hallucination/ not hallucination

                        [/INST] [/INST] Hallucination. The Generated output is not grammatically correct. The correct form would be "The children received presents." or "The children got presents."
<s>[INST] <s>[INST]An LLM was given the below Instructions to

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Can I find information about the code's approach to handling long-running tasks and background jobs?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])